# Modeling With Transformations

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

## Import Some Data

In [2]:
dataUrl = "https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv"
dataFromWeb = pd.read_csv(dataUrl)
dataFromWeb.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
labelField = 'charges'
featureData = dataFromWeb.drop(labelField, axis=1)
labelData = dataFromWeb[labelField]

## Split into Training & Testing Data

In [4]:
testDataPercentage = .2 # how much of our data should we use for "testing"
randomVal = 42
feature_training_data, feature_testing_data, label_training_data, label_testing_data = train_test_split(featureData, 
                                                    labelData, 
                                                    test_size=testDataPercentage, 
                                                    random_state=randomVal) # set random state for reproducible splits

## Transform
The `make_column_transformer` ([docs](https://scikit-learn.org/stable/modules/generated/sklearn.compose.make_column_transformer.html)) function can take a list of transformer functions along with a list of columns to apply the transformer to. This creates a transformer instance.  
The transformer instance, then, gets fitted to the data with the `fit` method.  
The transformer instnace, then, gets used with the data with the `transform` method.  
Here will be applied two transformers:
- the `MinMaxScaler`([docs](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)), to scale the `age`, `bmi`, and `children` column values to values between 0-and-1
- the `OneHotEncoder`([docs](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)), to ohe the `sex`, `smoker`, and `region` values

In [5]:
dataTransformer = make_column_transformer(
    # get all values between 0 and 1
    (MinMaxScaler(), ["age", "bmi", "children"]),
    (OneHotEncoder(handle_unknown="ignore"), ["sex", "smoker", "region"])
)

dataTransformer.fit(feature_training_data)

ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                 ['age', 'bmi', 'children']),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['sex', 'smoker', 'region'])])

## Normalize

In [6]:
normailized_feature_training_data = dataTransformer.transform(feature_training_data)
normailized_feature_testing_data = dataTransformer.transform(feature_testing_data)

### Compare normalized vs non-normalized

In [7]:
normailized_feature_training_data[0]

array([0.60869565, 0.10734463, 0.4       , 1.        , 0.        ,
       1.        , 0.        , 0.        , 1.        , 0.        ,
       0.        ])

In [8]:
feature_training_data.loc[0]

age                19
sex            female
bmi              27.9
children            0
smoker            yes
region      southwest
Name: 0, dtype: object

In [9]:
normailized_feature_training_data.shape

(1070, 11)

In [10]:
feature_training_data.shape

(1070, 6)

## Build A Model
This will be based on the `insurance_model_2` model that can be found in the `modeling-and-wrangling` notebook.  
This model version, though, will use normalized data: one-hot-encoded column values and scaled column values.

In [11]:
tf.random.set_seed(42)
m = tf.keras.Sequential()
epochs = 100
# different & more layers
l1 = tf.keras.layers.Dense(100)
l2 = tf.keras.layers.Dense(10)
l3 = tf.keras.layers.Dense(1)

m.add(l1)
m.add(l2)
m.add(l3)

# Compile the model
m.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=['mae'])

# Fit the model and save the history (we can plot this)
m_history = m.fit(normailized_feature_training_data, label_training_data, epochs=epochs, verbose=0)

### Review The Model

In [12]:
m.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               1200      
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2221 (8.68 KB)
Trainable params: 2221 (8.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
m.evaluate(normailized_feature_testing_data,label_testing_data)

9/9 [==============================] - 0s 5ms/step - loss: 3437.7786 - mae: 3437.7786


[3437.778564453125, 3437.778564453125]

In [14]:
print(f'Training Label Median: {label_training_data.median()}')
print(f'Training Label Mean: {label_training_data.mean()}')
print(f'm MAE: {m.get_metrics_result()["mae"].numpy()}')

Training Label Median: 9575.4421
Training Label Mean: 13346.089736364485
m MAE: 3437.778564453125


Compare this model mae to the `insurance_model_2` (_im2_) model in `modeling-and-wrangling`:
- `im2` had an MAE of `~4700`
- the new model mae looks to be `~3400`

**Normalizing this model's data, with one-hot-encoding and scaling, made this model perform better!**

## Experiment With The Model
### Double The Epochs

In [15]:
m2 = tf.keras.Sequential()
m2epochs = 200

m2.add(l1)
m2.add(l2)
m2.add(l3)

# Compile the model
m2.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=['mae'])

# Fit the model and save the history (we can plot this)
m2_history = m2.fit(normailized_feature_training_data, label_training_data, epochs=m2epochs, verbose=0)

#### Review The Model

In [16]:
m2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               1200      
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2221 (8.68 KB)
Trainable params: 2221 (8.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
m2.evaluate(normailized_feature_testing_data,label_testing_data)

9/9 [==============================] - 0s 5ms/step - loss: 3163.1138 - mae: 3163.1138


[3163.11376953125, 3163.11376953125]

In [24]:
print(f'Training Label Median: {label_training_data.median()}')
print(f'Training Label Mean: {label_training_data.mean()}')
print(f'm2 MAE: {m2.get_metrics_result()["mae"].numpy()}')
print(f'SHAPE: {normailized_feature_training_data.shape}')

Training Label Median: 9575.4421
Training Label Mean: 13346.089736364485
m2 MAE: 3163.11376953125
SHAPE: (1070, 11)


Increasing the Epochs _?slightly?_ made a positive impact on reducing the `mae`!

### Add A Layer, Change Layer Values

In [27]:
m3 = tf.keras.Sequential()

l4 = tf.keras.layers.Dense(100)

m3.add(l1)
m3.add(l4)
m3.add(l2)
m3.add(l3)

# Compile the model
m3.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=['mae'])

# Fit the model and save the history (we can plot this)
# , verbose=0
m3_history = m3.fit(normailized_feature_training_data, label_training_data, epochs=m2epochs)

Epoch 1/200
34/34 [==============================] - 3s 6ms/step - loss: 4890.2881 - mae: 4890.2881
Epoch 2/200
34/34 [==============================] - 0s 6ms/step - loss: 3555.6372 - mae: 3555.6372
Epoch 3/200
34/34 [==============================] - 0s 6ms/step - loss: 3499.9043 - mae: 3499.9043
Epoch 4/200
34/34 [==============================] - 0s 6ms/step - loss: 3509.0237 - mae: 3509.0237
Epoch 5/200
34/34 [==============================] - 0s 6ms/step - loss: 3540.9521 - mae: 3540.9521
Epoch 6/200
34/34 [==============================] - 0s 6ms/step - loss: 3517.5859 - mae: 3517.5859
Epoch 7/200
34/34 [==============================] - 0s 6ms/step - loss: 3518.6990 - mae: 3518.6990
Epoch 8/200
34/34 [==============================] - 0s 6ms/step - loss: 3529.7898 - mae: 3529.7898
Epoch 9/200
34/34 [==============================] - 0s 6ms/step - loss: 3534.4360 - mae: 3534.4360
Epoch 10/200
34/34 [==============================] - 0s 6ms/step - loss: 3517.0488 - mae: 3517.0488

#### Review The Model

In [28]:
m3.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               1200      
                                                                 
 dense_7 (Dense)             (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 12321 (48.13 KB)
Trainable params: 12321 (48.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
m3.evaluate(normailized_feature_testing_data,label_testing_data)

9/9 [==============================] - 0s 6ms/step - loss: 3223.4221 - mae: 3223.4221


[3223.422119140625, 3223.422119140625]

In [30]:
print(f'Training Label Median: {label_training_data.median()}')
print(f'Training Label Mean: {label_training_data.mean()}')
print(f'm3 MAE: {m3.get_metrics_result()["mae"].numpy()} vs m2 MAE: {m2.get_metrics_result()["mae"].numpy()}')

Training Label Median: 9575.4421
Training Label Mean: 13346.089736364485
m3 MAE: 3223.422119140625 vs m2 MAE: 3163.11376953125


Adding a layer made the mae roughly the same

### Change the Learning Rate less epochs

In [32]:
m4 = tf.keras.Sequential()

# l4 = tf.keras.layers.Dense(100)

m4.add(l1)
# m4.add(l4)
m4.add(l2)
m4.add(l3)

# Compile the model
m4.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(learning_rate=.008),
                          metrics=['mae'])

# Fit the model and save the history (we can plot this)
# , verbose=0
m4_history = m4.fit(normailized_feature_training_data, label_training_data, epochs=epochs)

Epoch 1/100
34/34 [==============================] - 3s 6ms/step - loss: 3510.7605 - mae: 3510.7605
Epoch 2/100
34/34 [==============================] - 0s 6ms/step - loss: 3485.2773 - mae: 3485.2773
Epoch 3/100
34/34 [==============================] - 0s 5ms/step - loss: 3482.4819 - mae: 3482.4819
Epoch 4/100
34/34 [==============================] - 0s 6ms/step - loss: 3489.0757 - mae: 3489.0757
Epoch 5/100
34/34 [==============================] - 0s 6ms/step - loss: 3488.5991 - mae: 3488.5991
Epoch 6/100
34/34 [==============================] - 0s 6ms/step - loss: 3494.5928 - mae: 3494.5928
Epoch 7/100
34/34 [==============================] - 0s 5ms/step - loss: 3487.7686 - mae: 3487.7686
Epoch 8/100
34/34 [==============================] - 0s 5ms/step - loss: 3485.9607 - mae: 3485.9607
Epoch 9/100
34/34 [==============================] - 0s 5ms/step - loss: 3488.7549 - mae: 3488.7549
Epoch 10/100
34/34 [==============================] - 0s 6ms/step - loss: 3489.1785 - mae: 3489.1785

#### Review The Model

In [28]:
m4.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               1200      
                                                                 
 dense_7 (Dense)             (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 12321 (48.13 KB)
Trainable params: 12321 (48.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
m4.evaluate(normailized_feature_testing_data,label_testing_data)

9/9 [==============================] - 0s 6ms/step - loss: 3223.4221 - mae: 3223.4221


[3223.422119140625, 3223.422119140625]

In [33]:
print(f'Training Label Median: {label_training_data.median()}')
print(f'Training Label Mean: {label_training_data.mean()}')
print(f'm3 MAE: {m4.get_metrics_result()["mae"].numpy()} vs m2 MAE: {m2.get_metrics_result()["mae"].numpy()}')

Training Label Median: 9575.4421
Training Label Mean: 13346.089736364485
m3 MAE: 3494.249755859375 vs m2 MAE: 3163.11376953125


setting the learning rate to `.008`, compared to the default `.001`, made the outcome slightly worse here :/ 

### Change the Learning Rate Again

In [34]:
m5 = tf.keras.Sequential()

# l4 = tf.keras.layers.Dense(100)

m5.add(l1)
# m5.add(l4)
m5.add(l2)
m5.add(l3)

# Compile the model
m5.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(learning_rate=.01),
                          metrics=['mae'])

# Fit the model and save the history (we can plot this)
# , verbose=0
m5_history = m5.fit(normailized_feature_training_data, label_training_data, epochs=epochs)

Epoch 1/100
34/34 [==============================] - 3s 6ms/step - loss: 3540.1484 - mae: 3540.1484
Epoch 2/100
34/34 [==============================] - 0s 5ms/step - loss: 3499.3572 - mae: 3499.3572
Epoch 3/100
34/34 [==============================] - 0s 6ms/step - loss: 3488.2312 - mae: 3488.2312
Epoch 4/100
34/34 [==============================] - 0s 6ms/step - loss: 3494.9104 - mae: 3494.9104
Epoch 5/100
34/34 [==============================] - 0s 6ms/step - loss: 3491.4202 - mae: 3491.4202
Epoch 6/100
34/34 [==============================] - 0s 5ms/step - loss: 3497.7678 - mae: 3497.7678
Epoch 7/100
34/34 [==============================] - 0s 6ms/step - loss: 3490.1836 - mae: 3490.1836
Epoch 8/100
34/34 [==============================] - 0s 5ms/step - loss: 3487.2910 - mae: 3487.2910
Epoch 9/100
34/34 [==============================] - 0s 5ms/step - loss: 3491.6929 - mae: 3491.6929
Epoch 10/100
34/34 [==============================] - 0s 5ms/step - loss: 3495.8262 - mae: 3495.8262

#### Review The Model

In [35]:
m5.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               1200      
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2221 (8.68 KB)
Trainable params: 2221 (8.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
m5.evaluate(normailized_feature_testing_data,label_testing_data)

9/9 [==============================] - 0s 6ms/step - loss: 3223.4221 - mae: 3223.4221


[3223.422119140625, 3223.422119140625]

In [36]:
print(f'Training Label Median: {label_training_data.median()}')
print(f'Training Label Mean: {label_training_data.mean()}')
print(f'm5 MAE: {m5.get_metrics_result()["mae"].numpy()} vs m2 MAE: {m2.get_metrics_result()["mae"].numpy()}')

Training Label Median: 9575.4421
Training Label Mean: 13346.089736364485
m5 MAE: 3509.501220703125 vs m2 MAE: 3163.11376953125


setting the learning rate to `.01`, compared to the default `.001`, made the outcome slightly worse here